# Fitting and Evaluating Models
Based on the small number of variables and the fact that this is a regression problem, I decided to keep the models fairly simple. In the pairplots in the [EDA and cleaning notebook](./02_eda_and_cleaning.ipynb), I observed what appeared to be a polynomial relationship between the magnitudes in each band (U, G, R, I, Z) and the redshift, so I tested a polynomial regression model.

In [1]:
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
# import modules for preprocessing and modeling

In [2]:
galaxies = pd.read_csv("../data/sdss_clean.csv", index_col=False)
# read in cleaned data

In [3]:
X = galaxies[["u", "g", "r", "i", "z"]]
y = galaxies["redshift"]
# create feature matrix and target vector

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20200310)
# split X and y into training and test sets for model fitting and evaluation

### Multiple Linear Regression

In [5]:
y_train_mean = y_train.mean()
y_train_baseline = list()
for i in range(len(y_train)):
    y_train_baseline.append(y_train_mean)

y_test_mean = y_test.mean()
y_test_baseline = list()
for i in range(len(y_test)):
    y_test_baseline.append(y_test_mean)

baseline_train_rmse = np.sqrt(MSE(y_train, y_train_baseline))
baseline_test_rmse = np.sqrt(MSE(y_test, y_test_baseline))

print(f"Training set baseline (RMSE): {baseline_train_rmse}")
print(f"Test set baseline (RMSE): {baseline_test_rmse}")
# for both training and test set: obtain mean squared error for a model that predicts the mean of y for every observation, then take the square root
# to obtain the RMSE and use this as the baseline score

Training set baseline (RMSE): 0.2226269235953162
Test set baseline (RMSE): 0.2244908042024483


In [6]:
linreg = LinearRegression()
linreg.fit(X_train, y_train);
# instantiate a linear regression and fit it to X and y

In [7]:
print(f"Training set r2 score: {cross_val_score(linreg, X_train, y_train).mean()}")
print(f"Test set r2 score: {cross_val_score(linreg, X_test, y_test).mean()}")
print(f"Training set RMSE: {np.sqrt(MSE(y_train, linreg.predict(X_train)))}")
print(f"Test set RMSE: {np.sqrt(MSE(y_test, linreg.predict(X_test)))}")
# get the average R2 score on the training and test set using 5-fold cross validation as well as RMSE

Training set r2 score: 0.8392073178821228
Test set r2 score: 0.8315979055902201
Training set RMSE: 0.08919968190914797
Test set RMSE: 0.09201922483354388


### Quadratic Regression

In [8]:
quad_poly = PolynomialFeatures(2)
# instantiate a polynomial features transformer

In [9]:
X_train_quad = quad_poly.fit_transform(X_train)
X_test_quad = quad_poly.transform(X_test)
# fit to and transform the training set features; transform the test set features

In [10]:
linreg_2 = LinearRegression()
linreg_2.fit(X_train_quad, y_train);
# instantiate a new linear regression and fit it to the transformed training set features

In [11]:
print(f"Training set r2 score: {cross_val_score(linreg_2, X_train_quad, y_train).mean()}")
print(f"Test set r2 score: {cross_val_score(linreg_2, X_test_quad, y_test).mean()}")
print(f"Training set RMSE: {np.sqrt(MSE(y_train, linreg_2.predict(X_train_quad)))}")
print(f"Test set RMSE: {np.sqrt(MSE(y_test, linreg_2.predict(X_test_quad)))}")

Training set r2 score: 0.8577803027139433
Test set r2 score: 0.8538380742950643
Training set RMSE: 0.08318758776911535
Test set RMSE: 0.0857170064855389


### Cubic Regression

In [12]:
cube_poly = PolynomialFeatures(3)

In [13]:
X_train_cube = cube_poly.fit_transform(X_train)
X_test_cube = cube_poly.transform(X_test)

In [14]:
linreg_3 = LinearRegression()
linreg_3.fit(X_train_cube, y_train);

In [15]:
print(f"Training set r2 score: {cross_val_score(linreg_3, X_train_cube, y_train).mean()}")
print(f"Test set r2 score: {cross_val_score(linreg_3, X_test_cube, y_test).mean()}")
print(f"Training set RMSE: {np.sqrt(MSE(y_train, linreg_3.predict(X_train_cube)))}")
print(f"Test set RMSE: {np.sqrt(MSE(y_test, linreg_3.predict(X_test_cube)))}")

Training set r2 score: 0.8033156511454067
Test set r2 score: 0.8381467722433819
Training set RMSE: 0.07914729674451988
Test set RMSE: 0.084846273370319


In [16]:
print(X_train.shape[1])
print(X_train_quad.shape[1])
print(X_train_cube.shape[1])
# number of features used for linear, quadratic and cubic regression

5
21
56


I believe that the quadratic regression is the strongest of these three models. It outperformed the linear regression on both metrics; the cubic regression saw a very slight reduction in RMSE over the quadratic, but this was offset by a reduction in r<sup>2</sup> scores as well as a large increase in the number of features.

### K Nearest Neighbors

In [17]:
scaler = StandardScaler()
# instantiate transformer to scale features

In [18]:
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)
# scale training set and test set features to enable KNN model to make good predictions

In [19]:
knn = KNeighborsRegressor()
# instantiate KNN regression with default 5 neighbors used
knn.fit(X_train_sc, y_train);
print(f"Training set R2 score: {cross_val_score(knn, X_train_sc, y_train).mean()}")
print(f"Test set R2 score: {cross_val_score(knn, X_test_sc, y_test).mean()}")
print(f"Training set RMSE: {np.sqrt(MSE(y_train, knn.predict(X_train_sc)))}")
print(f"Test set RMSE: {np.sqrt(MSE(y_test, knn.predict(X_test_sc)))}")
# print r2 and RMSE on training and test sets

Training set R2 score: 0.8674301716149604
Test set R2 score: 0.8518395208802725
Training set RMSE: 0.06552448183077234
Test set RMSE: 0.0823038157725722


In [20]:
r2_scores = list()
rmse = list()
# empty lists to hold evaluation metrics for 30 KNN models

for k in range(1, 31):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_sc, y_train);
    r2_scores.append((cross_val_score(knn, X_test_sc, y_test).mean(), k))
    rmse.append((np.sqrt(MSE(y_test, knn.predict(X_test_sc))), k))
    # instantiate and fit 30 KNN models with k=1 to k=30 neighbors; put the scores and k values in the lists

In [21]:
r2_scores.sort(reverse=True)
# sort r2 scores from highest to lowest

In [22]:
rmse.sort()
# sort RMSE values from lowest to highest

In [23]:
r2_scores[:5]
# see 5 best r2 scores

[(0.8588578414486603, 10),
 (0.8588010539775712, 8),
 (0.8587968020869482, 12),
 (0.8587603754409912, 9),
 (0.8587461021559388, 16)]

In [24]:
rmse[:5]
# see 5 best RMSE scores

[(0.08003349292320475, 15),
 (0.08009857182535854, 14),
 (0.08010869826868235, 17),
 (0.08012373861161977, 16),
 (0.0801550811947784, 13)]

k = 16 appears in both the 5 highest r<sup>2</sup> scores and the 5 lowest RMSE values. The KNN model using 16 neighbors gave an r<sup>2</sup> score of 0.859 and an RMSE of 0.080 on the test set. This is a very slight improvement over the quadratic regression. The fact that no values above 17 show up in either of these lists suggests that 16 is the optimal number, or at least a good tradeoff between performance and training time.

### Random Forest Regression

In [25]:
rfr = RandomForestRegressor(random_state=20200310)
# instantiate random forest regressor

In [26]:
rfr_params = {
    "n_estimators": [20, 30, 40],
    "max_depth": [8, 9, 10],
    "min_samples_split": [2, 10, 20]
}
# create dictionary of hyperparemeters for gridsearch
# limit number of estimators to 40  and max depth to 10 to minimize overfitting

In [27]:
gs = GridSearchCV(
    estimator=rfr,
    param_grid=rfr_params,
    return_train_score=True,
    n_jobs=6,
)
# instantiate gridsearch

In [28]:
gs.fit(X_train, y_train);
# run gridsearch to test models

In [29]:
results = pd.DataFrame(gs.cv_results_)
# create a DataFrame of the results from the gridsearch for ease of model selection

In [30]:
results.columns
# see what information is needed for model selection and what can be left out

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'param_min_samples_split', 'param_n_estimators',
       'params', 'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'split0_train_score',
       'split1_train_score', 'split2_train_score', 'split3_train_score',
       'split4_train_score', 'mean_train_score', 'std_train_score'],
      dtype='object')

In [31]:
reduced_results = results.drop(
    columns=[
        "mean_fit_time",
        "std_fit_time",
        "mean_score_time",
        "std_score_time",
        "params",
        "split0_test_score",
        "split1_test_score",
        "split2_test_score",
        "split3_test_score",
        "split4_test_score",
        "std_test_score",
        "split0_train_score",
        "split1_train_score",
        "split2_train_score",
        "split3_train_score",
        "split4_train_score",
        "std_train_score"
    ])
# drop the unnecessary columns and display the useful results

In [32]:
reduced_results.sort_values(by="rank_test_score")

,param_max_depth,param_min_samples_split,param_n_estimators,mean_test_score,rank_test_score,mean_train_score
26,10,20,40,0.879250,1,0.907124
23,10,10,40,0.879193,2,0.910157
25,10,20,30,0.878965,3,0.907034
20,10,2,40,0.878949,4,0.914200
22,10,10,30,0.878913,5,0.910040
19,10,2,30,0.878502,6,0.914051
21,10,10,20,0.878361,7,0.908932
24,10,20,20,0.878293,8,0.906053
18,10,2,20,0.877877,9,0.912810
17,9,20,40,0.875082,10,0.897076


Based on these results, it is worth running another gridsearch with larger values for min_samples_split while still limiting max_depth to 10 and n_estimators to 40 to control overfitting.

In [33]:
rfr = RandomForestRegressor(random_state=20200310)
rfr_params = {
    "n_estimators": [40],
    "max_depth": [10],
    "min_samples_split": list(range(10, 110, 10))
}
gs = GridSearchCV(
    estimator=rfr,
    param_grid=rfr_params,
    return_train_score=True,
    n_jobs=6,
)
gs.fit(X_train, y_train);
# instantiate and run another gridsearch with the new hyperparameters

In [34]:
results = pd.DataFrame(gs.cv_results_)
reduced_results = results.drop(
    columns=[
        "mean_fit_time",
        "std_fit_time",
        "mean_score_time",
        "std_score_time",
        "params",
        "split0_test_score",
        "split1_test_score",
        "split2_test_score",
        "split3_test_score",
        "split4_test_score",
        "std_test_score",
        "split0_train_score",
        "split1_train_score",
        "split2_train_score",
        "split3_train_score",
        "split4_train_score",
        "std_train_score"
    ])
reduced_results
# display the results

,param_max_depth,param_min_samples_split,param_n_estimators,mean_test_score,rank_test_score,mean_train_score
0,10,10,40,0.879193,2,0.910157
1,10,20,40,0.879250,1,0.907124
2,10,30,40,0.879032,3,0.905125
3,10,40,40,0.878843,4,0.903404
4,10,50,40,0.878486,5,0.901840
5,10,60,40,0.878295,6,0.900350
6,10,70,40,0.877906,7,0.898924
7,10,80,40,0.877590,8,0.897725
8,10,90,40,0.877137,9,0.896466
9,10,100,40,0.876829,10,0.895444


In [35]:
final_model = RandomForestRegressor(n_estimators=40, max_depth=10, min_samples_split=20, n_jobs=6, random_state=20200310)
final_model.fit(X_train, y_train);
# train a random forest model with the optimal hyperparameters on the training data

In [38]:
np.sqrt(MSE(y_test, final_model.predict(X_test)))
# obtain RMSE for the model

0.0797164356820923